In [1]:
from readlif.reader import LifFile
import os
import numpy as np
from tifffile import imwrite
import napari

### Pathways

In [2]:
# lif file with organoid movies
file_path  = r'R:\Aidan\ULA_timecourse_ARC_006.lif'

# temporary output file
dir_save = r'R:\Aidan\ULA_timecourse_ARC_006_temp'
os.makedirs(dir_save, exist_ok=True)

# choose bit depth for saving the tiff files
BIT_DEPTH = np.uint16

### Look inside the file

In [3]:
lif_file = LifFile(file_path)

im_list = [(ind,x['name'],x['dims'], x['channels']) for ind,x in enumerate(lif_file.image_list)]
im_list

[(0, 'TileScan 1/C/0944P', Dims(x=2048, y=2048, z=12, t=28, m=25), 2),
 (1, 'TileScan 1/C/1340P', Dims(x=2048, y=2048, z=12, t=28, m=25), 2),
 (2, 'TileScan 1/C/1341P', Dims(x=2048, y=2048, z=12, t=28, m=25), 2),
 (3, 'TileScan 1/C/0944P_Merged', Dims(x=10080, y=10187, z=12, t=28, m=1), 2),
 (4, 'TileScan 1/D/0944S', Dims(x=2048, y=2048, z=12, t=28, m=25), 2),
 (5, 'TileScan 1/D/1340S', Dims(x=2048, y=2048, z=12, t=28, m=25), 2),
 (6, 'TileScan 1/D/1341S', Dims(x=2048, y=2048, z=12, t=28, m=25), 2)]

In [4]:
########################################################################################
# specify which part of the name will be used for naming the tiles
# this part should be unique
########################################################################################

NAME_PARTS = [2]

### Specify files to process

In [7]:
#########################################################################################
# to process all available images
#########################################################################################
sel_ind = range(len(im_list)) # for all available images


#########################################################################################
# uncomment and change im_list if you want to run only on selected images
#########################################################################################
#sel_ind = [0,1,2,4,5,6] # for selected images

im_list_sel = [lif_file.get_image(ind) for ind in sel_ind]
im_list_sel

['LifImage object with dimensions: Dims(x=2048, y=2048, z=12, t=28, m=25)']

### Checks of the data

In [8]:
im_tile_list = []

for im in im_list_sel:

    if not int(np.sqrt(im.dims.m)) == np.sqrt(im.dims.m):
        print(f'{im.name} is not a full matrix, you need to add missing tiles.')
    else:
        print(f'{im.name} of dimensions {im.dims} appear to be a square.')

    im_tile = '_'.join(im.name.split('/')[i] for i in NAME_PARTS)
    im_tile_list.append(im_tile)
    print(f"These tiles will be saved as {im_tile}")

if len(im_tile_list) == len(set(im_tile_list)):

    print(f'All names are unique.')

else:

    print(f'Names are not unique - correct!')

TileScan 1/C/0944P of dimensions Dims(x=2048, y=2048, z=12, t=28, m=25) appear to be a square.
These tiles will be saved as 0944P
All names are unique.


### Process images

In [9]:
# specify missing tiles if a matrix is not square
# ex. if m=21 then additional_frames = [2,17,20]
# if a matrix is square leave additional_frames = []

additional_frames = []

In [10]:
for im in im_list_sel:

    print(f'Processing image: {im.name}')

    for ch in range(im.channels):

        print(f'Processing channel: {ch}')

        # that has to be recalculated because m and t are swapped
        t_search = 0
        m_search = 0

        for t in range (im.dims.t):

            print(f'Processing frame: {t}')

            frame_projections_list = []
            if len(additional_frames) > 0:
                frame_projection = np.zeros((im.dims.y,im.dims.x),dtype=BIT_DEPTH)
                frame_projections_list.append(frame_projection)

        
            for m in range(im.dims.m):

                frame_generator = im.get_iter_z(t = (t_search % im.dims.t), c=ch, m=m_search)

                t_search = t_search + 1
                m_search = t_search // im.dims.t

                frame_stack = np.zeros((im.dims.z,im.dims.y,im.dims.x),dtype=BIT_DEPTH)
                z = 0
                for frame in frame_generator:
                
                    frame_stack[z,:,:] = np.array(frame)
                    z += 1

                frame_projection = np.max(frame_stack,axis=0)

                frame_projections_list.append(frame_projection)

                if len(additional_frames) > 0:
                    if m in additional_frames:
                        frame_projection = np.zeros((im.dims.y,im.dims.x),dtype=BIT_DEPTH)
                        frame_projections_list.append(frame_projection)           


            for ind,frame in enumerate(frame_projections_list):

                # save the movie
                tile_name = '_'.join(im.name.split('/')[i] for i in NAME_PARTS)
                save_path = os.path.join(dir_save,tile_name + '_ch_' + str(ch).zfill(2) + '_m_' + str(ind).zfill(2) + '_t_' + str(t).zfill(3) + '.tif')
                imwrite(save_path, frame, imagej=True)

Processing image: TileScan 1/C/0944P
Processing channel: 0
Processing frame: 0
Processing frame: 1
Processing frame: 2
Processing frame: 3
Processing frame: 4
Processing frame: 5
Processing frame: 6
Processing frame: 7
Processing frame: 8
Processing frame: 9
Processing frame: 10
Processing frame: 11
Processing frame: 12
Processing frame: 13
Processing frame: 14
Processing frame: 15
Processing frame: 16
Processing frame: 17
Processing frame: 18
Processing frame: 19
Processing frame: 20
Processing frame: 21
Processing frame: 22
Processing frame: 23
Processing frame: 24
Processing frame: 25
Processing frame: 26
Processing frame: 27
Processing channel: 1
Processing frame: 0
Processing frame: 1
Processing frame: 2
Processing frame: 3
Processing frame: 4
Processing frame: 5
Processing frame: 6
Processing frame: 7
Processing frame: 8
Processing frame: 9
Processing frame: 10
Processing frame: 11
Processing frame: 12
Processing frame: 13
Processing frame: 14
Processing frame: 15
Processing fram